# Forest CoverType 2b): PyTorch TabNet

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os

# External Dependencies:
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator
from sagemaker.pytorch.model import PyTorchModel
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

# Local Dependencies:
import util

In [ ]:
%store -r bucket_name
%store -r experiment_name
%store -r preproc_trial_component_name

bucket = boto3.resource("s3").Bucket(bucket_name)
role = sagemaker.get_execution_role()
smclient = boto3.client("sagemaker")
smsess = sagemaker.session.Session()

In [ ]:
tabnet_trial = Trial.create(
    trial_name=util.append_timestamp("tabnet-pytorch"), 
    experiment_name=experiment_name,
    sagemaker_boto_client=smclient,
)
tabnet_trial.add_trial_component(preproc_trial_component_name)

In [ ]:
hyperparameters = {
    "model-type": "classification",
    "target": "Cover_Type",
    "seed": 1337,
    "n-d": 64,
    "n-a": 64,
    "n-steps": 5,
    "lr": 0.02,
    "gamma": 1.5,
    "n-independent": 2,
    "n-shared": 2,
    #"cat-idxs": ",".join(map(lambda i: str(i), cat_idxs)),
    # cat-dims???
    #"cat-emb-dim": ",".join(map(lambda i: str(i), cat_emb_dim)),
    "lambda-sparse": 1e-4,
    "momentum": 0.3,
    "clip-value": 2.,
    "max-epochs": 1000,
    "patience": 100,
    "batch-size": 16384,
    "virtual-batch-size": 256,
}


estimator = PyTorchEstimator(
    role=role,
    entry_point="train.py",
    source_dir="src",
    framework_version="1.4",

    base_job_name="forestcover-tabnet",

    debugger_hook_config=False,

    train_instance_count=1,
    train_instance_type="ml.p3.2xlarge",
    hyperparameters=hyperparameters,
    metric_definitions=[
        # One console log per output e.g.:
        # | EPOCH | train | valid | total time (s)
        # | 1 | 0.58782 | 0.06811 | 25.5
        # Since these rows are a bit brusque, we'll write quite precise/picky regexs to stay safe:
        # TODO: Extraction not working?
        { "Name": "train:accuracy", "Regex": r"\| +\d+ +\| +(.*?) +\| +[^\s]+ +\| +[^\s]+", },
        { "Name": "validation:accuracy", "Regex": r"\| +\d+ +\| +[^\s] +\| +(.*?)+ +\| +[^\s]+", },
    ],
    enable_sagemaker_metrics=True,
)

In [ ]:
estimator.fit(
    inputs={
        "train": f"s3://{bucket_name}/data/train.csv",
        "validation": f"s3://{bucket_name}/data/validation.csv",
    },
    experiment_config={
        # This will create a TrainingJob-linked TrialComponent and automatically attach hyperparameters etc
        "TrialName": tabnet_trial.trial_name,
        "TrialComponentDisplayName": "Training",
    },
    #wait=False,
)

## Deploy

In [ ]:
model_path = estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]
model = PyTorchModel(
    name="tabnet-3",
    model_data=model_path,
    role=role,
    source_dir="src/",
    entry_point="src/inference.py",
    framework_version="1.4"
)

predictor = model.deploy(
    endpoint_name="tabnet-3",
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    wait=False
)

## Test

In [ ]:
# TODO: Load df_raw_test
X_test = df_raw_test.drop("Cover_Type", axis=1).to_numpy()
X_test.shape

In [ ]:
import io

s = io.StringIO()
df_raw_test.drop("Cover_Type", axis=1)[0:10].to_csv(s, index=False, header=False)

In [ ]:
response = predictor.predict(df_raw_test.drop("Cover_Type", axis=1).iloc[0:10].to_numpy())
# yields numpy array of ints (no confidence scores!)